# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 45 new papers today
          18 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/17 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2210.11488


extracting tarball to tmp_2210.11488...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.11506


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2210.11488/menu.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'Section_5' from 'tmp_2210.11488/Section_5.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'Section_4' from 'tmp_2210.11488/Section_4.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_dec

...

extracting tarball to tmp_2210.11977...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.11986


extracting tarball to tmp_2210.11986...

 done.


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2210.11986/FMS_UHJ.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.11551-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.11551) | **Properties of atomic hydrogen gas in the Galactic plane from THOR 21-cm  absorption spectra: a comparison with the high latitude gas**  |
|| Arghyadeep Basu, et al. -- incl., <mark>Henrik Beuther</mark>, <mark>Jonas Syed</mark> |
|*Appeared on*| *2022-10-24*|
|*Comments*| *6 pages, 5 figures, submitted and accepted for publication in MNRAS*|
|**Abstract**| The neutral hydrogen 21 cm line is an excellent tracer of the atomic interstellar medium in the cold and the warm phases. Combined 21 cm emission and absorption observations are very useful to study the properties of the gas over a wide range of density and temperature. In this work, we have used 21 cm absorption spectra from recent interferometric surveys, along with the corresponding emission spectra from earlier single dish surveys to study the properties of the atomic gas in the Milky Way. In particular, we focus on a comparison of properties between lines of sight through the gas disk in the Galactic plane and high Galactic latitude lines of sight through more diffuse gas. As expected, the analysis shows a lower average temperature for the gas in the Galactic plane compared to that along the high latitude lines of sight. The gas in the plane also has a higher molecular fraction, showing a sharp transition and flattening in the dust - gas correlation. On the other hand, the observed correlation between 21 cm brightness temperature and optical depth indicates some intrinsic difference in spin temperature distribution and a fraction of gas in the Galactic plane having intermediate optical depth (for 0.02 < $\tau$ < 0.2) but higher spin temperature, compared to that of the diffuse gas at high latitude with the same optical depth. This may be due to a small fraction of cold gas with slightly higher temperature and lower density present on the Galactic plane. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.11926-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.11926) | **The interstellar medium distribution, gas kinematics, and system  dynamics of the far-infrared luminous quasar SDSS J2310+1855 at $z=6.0$**  |
|| Yali Shao, et al. -- incl., <mark>Fabian Walter</mark>, <mark>Frank Bertoldi</mark>, <mark>Linhua Jiang</mark> |
|*Appeared on*| *2022-10-24*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**| We present ALMA sub-kpc- to kpc-scale resolution observations of the [CII], CO(9-8), and OH$^{+}$\,($1_{1}$--$0_{1}$) lines along with their dust continuum emission toward the FIR luminous quasar SDSS J231038.88+185519.7 at $z = 6.0031$. The [CII] brightness follows a flat distribution with a Sersic index of 0.59. The CO(9-8) line and the dust continuum can be fit with an unresolved nuclear component and an extended Sersic component with a Sersic index of ~1. The dust temperature drops with distance from the center. The effective radius of the dust continuum is smaller than that of the line emission and the dust mass surface density, but is consistent with that of the star formation rate surface density. The OH$^{+}$\,($1_{1}$--$0_{1}$) line shows a P-Cygni profile with an absorption, which may indicate an outflow with a neutral gas mass of $(6.2\pm1.2)\times10^{8} M_{\odot}$ along the line of sight. We employed a 3D tilted ring model to fit the [CII] and CO(9-8) data cubes. The two lines are both rotation dominated and trace identical disk geometries and gas motions. We decompose the circular rotation curve measured from the kinematic model fit to the [CII] line into four matter components (black hole, stars, gas, and dark matter). The quasar-starburst system is dominated by baryonic matter inside the central few kiloparsecs. We constrain the black hole mass to be $2.97^{+0.51}_{-0.77}\times 10^{9}\,M_{\odot}$; this is the first time that the dynamical mass of a black hole has been measured at $z\sim6$. A massive stellar component (on the order of $10^{9}\,M_{\odot}$) may have already existed when the Universe was only ~0.93 Gyr old. The relations between the black hole mass and the baryonic mass of this quasar indicate that the central supermassive black hole may have formed before its host galaxy. [Abridged version. Please see the full abstract in the manuscript.] |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.11977-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.11977) | **The SOUX AGN sample: Optical/UV/X-ray SEDs and the nature of the disc**  |
|| Jake A. J. Mitchell, et al. -- incl., <mark>Hermine Landt</mark> |
|*Appeared on*| *2022-10-24*|
|*Comments*| *Submitted to MNRAS*|
|**Abstract**| We use the SOUX sample of $\sim$700 AGN to form average optical-UV-X-rays SEDs on a 2D grid of $M_{\mathrm{BH}}$ and $L_{2500}$. We compare these with the predictions of a new AGN SED model, QSOSED, which includes prescriptions for both hot and warm Comptonisation regions as well as an outer standard disc. This predicts the overall SED fairly well for 7.5<log($M_{\mathrm{BH}}/M_{\mathrm{\odot}}$)<9.0 over a wide range in $L/L_{\mathrm{Edd}}$, but at higher masses the outer disc spectra in the model are far too cool to match the data. We create optical-UV composites from the entire SDSS sample and use these to show that the mismatch is due to there being no significant change in spectral shape of the optical-UV continuum across several decades of $M_{\mathrm{BH}}$ at constant luminosity. We show for the first time that this cannot be matched by standard disc models with high black hole spin. These apparently fit, but are not self-consistent as they do not include the General Relativistic effects for the emission to reach the observer. At high spin, increased gravitational redshift compensates for almost all of the higher temperature emission from the smaller inner disc radii. The data do not match the predictions made by any current accretion flow model. Either the disc is completely covered by a warm Comptonisation layer whose properties change systematically with $L/L_{\mathrm{Edd}}$, or the accretion flow structure is fundamentally different to that of the standard disc models. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.11967-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.11967) | **The effect of water and electron collisions in the rotational excitation  of HF in comets**  |
|| J. Loreau, A. Faure, <mark>F. Lique</mark> |
|*Appeared on*| *2022-10-24*|
|*Comments*| *8 pages*|
|**Abstract**| We present the first set of rate coefficients for the rotational excitation of the 7 lowest levels of hydrogen fluoride (HF) induced by collision with water molecules, the dominant collider in cometary comas, in the 5-150 K temperature range. The calculations are performed with a quantum statistical approach from an accurate rigid rotor ab initio interaction potential. Rate coefficients for excitation of HF by electron-impact are also computed, within the Born approximation, in the 10-10,000 K temperature range. These rate coefficients are then used in a simplified non-local thermodynamic equilibrium (non-LTE) model of a cometary coma that also includes solar radiative pumping and radiative decay. We investigate the range of H2O densities that lead to non-LTE populations of the rotational levels of HF. We show that to describe the excitation of HF in comets, considering collisions with both water molecules and electrons is needed as a result of the large dipole of HF. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.11986-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.11986) | **The Mantis Network II: Examining the 3D high-resolution observable  properties of the UHJs WASP-121b and WASP-189b through GCM modelling**  |
|| <mark>Elspeth K. H. Lee</mark>, et al. -- incl., <mark>Kevin Heng</mark> |
|*Appeared on*| *2022-10-24*|
|*Comments*| *MNRAS accepted: 8 August 2022, published 11 August 2022*|
|**Abstract**| The atmospheres of ultra hot Jupiters (UHJs) are prime targets for the detection of molecules and atoms at both low and high spectral resolution. We study the atmospheres of the UHJs WASP-121b and WASP-189b by performing 3D general circulation models (GCMs) of these planets using high temperature correlated-k opacity schemes with ultra-violet (UV) absorbing species included. The GCM results are then post-processed at low and high spectral resolutions and compared to available data. The high resolution results are cross-correlated with molecular and atomic templates to produce mock molecular detections. Our GCM models produce similar temperature-pressure (T-p) structure trends to previous 1D radiative-convective equilibrium models of UHJs. Furthermore, the inclusion of UV opacities greatly shapes the thermal and dynamical properties of the high-altitude, low-pressure regions of the UHJ atmospheres, with sharp T-p inversions due to the absorption of UV light. This suggests that optical wavelength, high-resolution observations probe a dynamically distinct upper atmospheric region, rather than the deeper jet forming layers. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.11964-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.11964) | **Spiral arms and the angular momentum gap in Milky Way Cepheids**  |
|| Marcin Semczuk, <mark>Walter Dehnen</mark>, Ralph Schoenrich, E. Athanassoula |
|*Appeared on*| *2022-10-24*|
|*Comments*| *6 pages, 7 figures, submitted to MNRAS*|
|**Abstract**| The angular-momentum distribution of classical Cepheids in the outer Milky Way disc is bi-modal with a gap at $L_\mathrm{gap}=2950\,\mathrm{km}\,\mathrm{s}^{-1}\,\mathrm{kpc}$, corresponding to $R=13\,\mathrm{kpc}$, while no similar feature has been found in the general population of disc stars. We show that star formation in multiple spiral arm segments at the same azimuth leads to such multi-modality which quickly dissolves and only shows in young stars. Unlike other explanations, such as a 1:1 orbital resonance with the Galactic bar, this also accounts for the observed steepening of the stellar warp at $L_\mathrm{gap}$, since the adjacent spiral arms represent different parts of the warped gas disc. In this scenario the gap is clearly present only in young stars, as observed, while most purely stellar dynamical origins would affect all disc populations, including older disc stars. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Max' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.11878-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.11878) | **Metal-THINGS: The association and optical characterization of SNRs with  HI holes in NGC 6946**  |
|| <mark>M. A. Lara-Lopez</mark>, et al. |
|*Appeared on*| *2022-10-24*|
|*Comments*| *Accepted by A&A*|
|**Abstract**| NGC~6946, also known as the `Fireworks' galaxy, is an unusual galaxy that hosts a total of 225 supernova remnant (SNR) candidates, including 147 optically identified with high [SII]/Ha line ratios. In addition, this galaxy shows prominent HI holes, which were analyzed in previous studies. Indeed, the connection between SNRs and HI holes together with their physical implications in the surrounding gas is worth of attention. This paper explores the connection between the SNRs and the HI holes, including an analysis of their physical link to observational optical properties inside and around the rims of the holes, using new integral field unit (IFU) data from the Metal-THINGS survey. We present an analysis combining previously identified HI holes, SNRs candidates, and new integral field unit (IFU) data from Metal-THINGS of the spiral galaxy NGC 6946. We analyze the distributions of the oxygen abundance, star formation rate surface density, extinction, ionization, diffuse ionized gas, and the Baldwin-Phillips-Terlevich classification throughout the galaxy. By analyzing in detail the optical properties of the 121 previously identify HI holes in NGC 6946, we find that the SNRs are concentrated at the rims of the HI holes. Furthermore, our IFU data shows that the star formation rate and extinction are enhanced at the rims of the holes. To a lesser degree, the oxygen abundance and ionization parameter show hints of enhancement on the rims of the holes. Altogether, this provides evidence of induced star formation taking place at the rims of the holes, whose origin can be explained by the expansion of superbubbles created by multiple supernova explosions in large stellar clusters dozens of Myr ago. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error [Errno 2] No such file or directory: 'gs'</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.11600-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.11600) | **Statistical Study and Live Catalogue of Multi-Spacecraft 3He-Rich Time  Periods over Solar Cycles 23, 24, and 25**  |
|| Samuel T. Hart, et al. -- incl., <mark>Maher A. Dayeh</mark>, <mark>Robert W. Ebert</mark>, <mark>Gang Li</mark> |
|*Appeared on*| *2022-10-24*|
|*Comments*| *22 pages, 9 figures, publishing in ApJ Supplemental Series*|
|**Abstract**| Using ion measurements from Ultra-Low-Energy Isotope Spectrometer (ULEIS) observations onboard Advanced Composition Explorer (ACE) and Solar Isotope Spectrometer (SIS) observations onboard the Solar Terrestrial Observatory (STEREO)-A and STEREO-B spacecraft, we have identified 854 3He-rich time periods between 1997 September and 2021 March. We include all event types with observed 3He enhancements such as corotating interaction regions (CIRs), gradual solar energetic particle (SEP) events, interplanetary shocks, and impulsive SEP events. We employ two different mass separation techniques to obtain 3He, 4He, Fe, and O fluences for each event, and we determine the 3He/4He and Fe/O abundance ratios between 0.32 to 0.45 MeV/nucleon and 0.64 to 1.28 MeV/nucleon. We find a clear correlation in the 3He/4He and Fe/O abundance ratios between both energy ranges. We find two distinct trends in the 3He/4He vs. Fe/O relation. For low 3He/4He values, there is a positive linear correlation between 3He/4He and Fe/O. However, at 3He/4He ~ 0.3, Fe/O appears to reach a limit and the correlation weakens significantly. We provide a live catalogue of 3He rich time periods that includes the robust determination of the onset and end times of the 3He enhancements in SEP-associated periods for different types of events observed my multiple spacecraft. This catalogue is available for public use. New releases will follow after major additions such as adding new periods from new missions (e.g., Parker Solar Probe and Solar Orbiter), identifying event types (impulsive SEP events, etc.), or adding new parameters such as remote observations detailing characteristics of the active regions. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2210.11551.md
    + _build/html/tmp_2210.11551/./fig1a.png
    + _build/html/tmp_2210.11551/./fig1b.png
    + _build/html/tmp_2210.11551/./fig4a.png
    + _build/html/tmp_2210.11551/./fig4b.png
    + _build/html/tmp_2210.11551/./fig4c.png
    + _build/html/tmp_2210.11551/./fig5.png
exported in  _build/html/2210.11926.md
    + _build/html/tmp_2210.11926/./rcd.png
    + _build/html/tmp_2210.11926/./rcd2.png
    + _build/html/tmp_2210.11926/./J2310_withoutbh.png
    + _build/html/tmp_2210.11926/./sb_co98_model_fit_plot_final_mc_compare.png
    + _build/html/tmp_2210.11926/./J2310_split_12m+7m_cii_image_chan29-60_mom0.png
    + _build/html/tmp_2210.11926/./J2310_split_12m+7m_cii_image_chan29-60_4sigma_mom1.png
    + _build/html/tmp_2210.11926/./J2310_split_12m+7m_cii_image_chan29-60_mom0_2sigma_spectrum.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Properties of atomic hydrogen gas in the Galactic plane from THOR 21-cm absorption spectra: a comparison with the high latitude gas

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2210.11551-b31b1b.svg)](https://arxiv.org/abs/2210.11551)<mark>Appeared on: 2022-10-24</mark> - _6 pages, 5 figures, submitted and accepted for publication in MNRAS_

</div>
<div id="authors">

Arghyadeep Basu, et al. -- incl., <mark><mark>Henrik Beuther</mark></mark>, <mark><mark>Jonas Syed</mark></mark>

</div>
<div id="abstract">

**Abstract:** The neutral hydrogen 21 cm line is an excellent tracer of the atomic interstellar medium in the cold and the warm phases. Combined 21 cm emission and absorption observations are very useful to study the properties of the gas over a wide range of density and temperature. In this work, we have used 21 cm absorption spectra from recent interferometric surveys, along with the corresponding emission spectra from earlier single dish surveys to study the properties of the atomic gas in the Milky Way. In particular, we focus on a comparison of properties between lines of sight through the gas disk in the Galactic plane and high Galactic latitude lines of sight through more diffuse gas. As expected, the analysis shows a lower average temperature for the gas in the Galactic plane compared to that along the high latitude lines of sight. The gas in the plane also has a higher molecular fraction, showing a sharp transition and flattening in the dust - gas correlation. On the other hand, the observed correlation between 21 cm brightness temperature and optical depth indicates some intrinsic difference in spin temperature distribution and a fraction of gas in the Galactic plane having intermediate optical depth (for$0.02<\tau<0.2$) but higher spin temperature, compared to that of the diffuse gas at high latitude with the same optical depth. This may be due to a small fraction of cold gas with slightly higher temperature and lower density present on the Galactic plane.

</div>

<div id="div_fig1">

<img src="tmp_2210.11551/./fig1a.png" alt="Fig2.1" width="50%"/><img src="tmp_2210.11551/./fig1b.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Observed $T_{B} - \tau$ distribution (in $log_{10}$ scale) for H {\sc i} 21 cm emission and absorption lines. \textbf{Left}: Optical depth ($\tau$) from the THOR and brightness temperature $T_B$ from LAB survey (resampled to the same velocity resolution of 1.5 km s$^{-1}$) for corresponding spectral channels shown as density contours (red). The comparable $T_{B} - \tau$ distribution for the high latitude sample (ie. \citealt{saha}) is shown in green. Red and green dashed lines are corresponding fitted straight lines for these two samples. \textbf{Right:}(Top panel) Binned $T_B - \tau$ observed median distribution for low (red line) and high latitudes (green line) lines of sight. Dashed lines indicate the $T_B - \tau$ relation for constant $T_{s}$ values. (Bottom panel) Statistical values for the non-parametric K-S test over the same $log_{10}(\tau)$ range. (*fig:figure1*)

</div>
<div id="div_fig2">

<img src="tmp_2210.11551/./fig4a.png" alt="Fig5.1" width="33%"/><img src="tmp_2210.11551/./fig4b.png" alt="Fig5.2" width="33%"/><img src="tmp_2210.11551/./fig4c.png" alt="Fig5.3" width="33%"/>

**Figure 5. -** Comparison of H {\sc i} column density fraction in different $T_{s}$ ranges between the THOR sample (this work) and the high latitude sample \citep{roy,saha}, shown in red and blue line respectively. \textbf{Left panel}: Entire range of $\tau$; \textbf{Middle panel}: For $\tau > 0.02$\textbf{Right panel}: For $0.2 > \tau > 0.02$.  (*fig:tsdistribution*)

</div>
<div id="div_fig3">

<img src="tmp_2210.11551/./fig5.png" alt="Fig1" width="100%"/>

**Figure 1. -** Comparison of inferred $T_{k}$ and $T_{s}$ from the THOR data with models from \citet{Lizst}. Solid lines are model $T_{k} - T_{s}$ relation for different thermal pressure. Line with filled circles is from this analysis, for $P_{th} = 3000  K cm^{-3}$ and $l_c = 3  pc$. Dashed lines are for $l_c = 6$ and 12 pc. Line with empty squares is based on data from \citet{saha} for $l_c = 2  pc$ for a comparison with the low latitude sample. (*fig:tstkcompare*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# z=6.0

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2210.11926-b31b1b.svg)](https://arxiv.org/abs/2210.11926)<mark>Appeared on: 2022-10-24</mark> - _Accepted for publication in A&A_

</div>
<div id="authors">

Yali Shao, et al. -- incl., <mark>Fabian Walter</mark>, <mark>Frank Bertoldi</mark>, <mark>Linhua Jiang</mark>

</div>
<div id="abstract">

**Abstract:** We present Atacama Large Millimeter/submillimeter Array (ALMA)  sub-kiloparsec- to kiloparsec-scale resolution observations of the[$\ion${C}{II}], CO (9--8), and OH$^{+}$($1_{1}$--$0_{1}$)  lines along with their dust continuum emission toward the far-infrared (FIR) luminous quasar SDSS J231038.88+185519.7 at$z = 6.0031$, to study the interstellar medium distribution, the gas kinematics, and the quasar-host system dynamics. We decompose the intensity maps of the[$\ion${C}{II}]and CO (9--8) lines and  the dust continuum with two-dimensional elliptical S{\'e}rsic models. The[$\ion${C}{II}]brightness follows a flat distribution with a S{\'e}rsic index of 0.59. The CO (9--8) line and the dust continuum can be  fit with an unresolved nuclear component and an extended S{\'e}rsic component with a S{\'e}rsic index of$\sim$1, which may correspond to the emission from an active galactic nucleus dusty molecular torus and a quasar host galaxy, respectively. The different[$\ion${C}{II}]spatial distribution may be due to the effect of the high dust opacity, which increases the FIR background radiation on the[$\ion${C}{II}]line, especially in the galaxy center, significantly suppressing the[$\ion${C}{II}]emission profile. The dust temperature drops with distance from the center. The effective radius of the dust continuum is smaller than that of the line emission and the dust mass  surface density, but is consistent with that of the star formation rate surface density. This may indicate that the dust emission is a less robust tracer of the dust and gas distribution but is a decent tracer of the obscured star formation activity. The OH$^{+}$($1_{1}$--$0_{1}$) line shows a P-Cygni profile with an absorption at$\sim$--400 km/s, which may indicate an outflow with a neutral gas mass of$(6.2\pm1.2)\times10^{8} M_{\odot}$along the line of sight. We employed a three-dimensional tilted ring model to fit the[$\ion${C}{II}]and CO (9--8)  data cubes. The two lines   are both rotation dominated and trace identical disk geometries and gas motions. This suggest that the[$\ion${C}{II}]and CO (9--8) gas are coplanar and corotating in this quasar host galaxy. The consistent circular velocities measured with[$\ion${C}{II}]and CO (9--8)  lines indicate that these two lines trace a similar gravitational potential. We decompose the circular rotation curve measured from the kinematic model fit to the[$\ion${C}{II}]line into four matter components (black hole, stars, gas, and dark matter). The quasar-starburst system is dominated by  baryonic matter inside the central few kiloparsecs. We constrain the black hole mass to be$2.97^{+0.51}_{-0.77}\times 10^{9} M_{\odot}$; this is the first time that the dynamical mass of a black hole has been measured at$z\sim6$. This mass is consistent with that determined using the scaling relations from quasar emission lines. A massive stellar component (on the order of$10^{9} M_{\odot}$) may have already existed when the Universe was only$\sim$0.93 Gyr old. The relations between the black hole mass and  the baryonic mass of this quasar indicate that the central supermassive black hole may have formed before its host galaxy.

</div>

<div id="div_fig1">

<img src="tmp_2210.11926/./rcd.png" alt="Fig14.1" width="33%"/><img src="tmp_2210.11926/./rcd2.png" alt="Fig14.2" width="33%"/><img src="tmp_2210.11926/./J2310_withoutbh.png" alt="Fig14.3" width="33%"/>

**Figure 14. -** Best-fit of the decomposition of the circular rotation curve traced by the [$\ion${C}{II}] line when allowing the black hole component to be free (left), fixed to $1.8\times10^{9} M_{\odot}$(\citealt{Feruglio2018}; middle) and none (right). Black points with error bars are the circular velocities, these  are the rotation velocities corrected for the asymmetric drift (caused by  gas random motions). The solid yellow, green, brown and blue lines represent the black hole, stellar, gas and dark matter components, respectively. The dashed red lines are the sum of these four components. The blue diamonds with error bars are the circular velocities measured from the  CO (9--8) line. These two lines trace the identical gravitational potential within the errors. (*rcd*)

</div>
<div id="div_fig2">

<img src="tmp_2210.11926/./sb_co98_model_fit_plot_final_mc_compare.png" alt="Fig9" width="100%"/>

**Figure 9. -**  Image decomposition of the CO (9--8) line. Top-left panels: 2D elliptical S{\'e}rsic model and the residual between the observed and modeled CO (9--8) intensity maps, from left to right. We measured the residual rms inside the dashed gray square with a side length of 1$\arcsec$. Bottom-left panels: Same as in the top panels but for the Point+2D elliptical S{\'e}rsic modeling. The peak values and rms within the dashed gray square are 0.061 and 0.021, and 0.059 and 0.020 Jy/beam km/s for the S{\'e}rsic and P+S{\'e}rsic residual maps, respectively. The shape of the CO (9--8) synthesized beam  with a FWHM size of $0$\farcs$187\times0$\farcs$153$ is plotted in the bottom-left corner of each panel. Top-right panel: Distributions of the residuals for pixels inside the dashed gray squares (S{\'e}rsic modeling: blue histogram; P+S{\'e}rsic modeling: red histogram). Middle-right panel: CO (9--8) luminosity surface brightness (black squares, blue circles, and red diamonds with error bars are measurements from the observed intensity map, the 2D elliptical S{\'e}rsic model, and the Point+2D elliptical S{\'e}rsic model, respectively) at different radii, measured using elliptical rings with the ring width along the major axis half ($0$\farcs$1$) that of the CO (9--8) clean beam size, the rotation angle equal to $\overline{\rm PA}$(=198$\degr$), and the ratio of semiminor and semimajor axis -- $b/a$ of $\cos(\overline{i})$($\overline{i}=43\degr$), where $\overline{\rm PA}$ and $\overline{i}$ come from the CO (9--8) line kinematic modeling (listed in Table \ref{par_kin}). The error bar represents the deviation of the values of all pixels in each ring. The solid and dashed gray lines are three times the CO (9--8) luminosity surface brightness limit measured in the emission-free region with an elliptical annulus that is the same with the ones used to measure the CO (9--8) luminosity surface brightness but has a larger radius, and its corresponding rms. Bottom-right panel: Luminosity surface brightness difference between the measurements from the observed intensity map and the ones from the 2D elliptical S{\'e}rsic model (blue circles) and from the Point+2D elliptical S{\'e}rsic model (red diamonds). Note that the vertical scale is linear and in units of $10^{6}L_{\odot}/\rm kpc^{2}$. (*sbco98*)

</div>
<div id="div_fig3">

<img src="tmp_2210.11926/./J2310_split_12m+7m_cii_image_chan29-60_mom0.png" alt="Fig4.1" width="33%"/><img src="tmp_2210.11926/./J2310_split_12m+7m_cii_image_chan29-60_4sigma_mom1.png" alt="Fig4.2" width="33%"/><img src="tmp_2210.11926/./J2310_split_12m+7m_cii_image_chan29-60_mom0_2sigma_spectrum.png" alt="Fig4.3" width="33%"/>

**Figure 4. -** 
ALMA observed [$\ion${C}{II}] line. Left panel: [$\ion${C}{II}] velocity-integrated map. The white plus sign is the \textit{Hubble} Space Telescope (HST) quasar position (RA of 23:10:38.90; Dec of 18:55:19.85). The shape of the restoring beam with a FWHM size of $0$\farcs$111$$\times$$0$\farcs$092$ is plotted  in the bottom-left corner. The contour levels are  [$-$3, 3, 6, 9, 12, 15]$\times$ rms$_{\rm[$\ion${C}{II}]}$, where rms$_{\rm[$\ion${C}{II}]}=0.035$ Jy/beam km/s.
Middle panel: [$\ion${C}{II}] velocity map produced using pixels above 4$\sigma$ in the channel maps. The black plus sign is the HST quasar position. A clear velocity gradient can be seen.
Right panel: [$\ion${C}{II}] line spectrum (black histogram) overplotted with the best-fit Gaussian profiles. The spectrum is extracted from the 2$\sigma$ region of the source emitting area in the [$\ion${C}{II}] intensity map. The solid blue line presents the best-fit single Gaussian model. The solid red line represents the best-fit double-Gaussian model, and the dashed and dotted red lines are for each of the double Gaussians. The  kinematic local standard of rest (LSRK) velocity scale is relative to the [$\ion${C}{II}] redshift from our ALMA Cycle 0 observations \citep{Wang2013}. The spectral resolution is 15.625 MHz, corresponding to 17 km/s. The rms of the spectrum measured using the line-free spectrum is 0.65 mJy and is shown as a black bar. The  [$\ion${C}{II}] spectral profile is asymmetric, with enhancement on the red side. The green bar indicates the velocity range of the OH$^{+}$ absorption.
 (*ciimap*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

72  publications in the last 7 days.
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers